In [1]:
import keras
from keras import layers
import numpy as np
import os
from keras.datasets import cifar10
from keras.preprocessing import image

Using TensorFlow backend.
C:\Users\lucif\Anaconda3\envs\TF\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\lucif\Anaconda3\envs\TF\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\lucif\Anaconda3\envs\TF\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\lucif\Anaconda3\envs\TF\lib\site-packages\tenso

In [2]:
latent_dim=32
height=32
width=32
channels=3

generator_input=keras.Input(shape=(latent_dim,))
x=layers.Dense(128*16*16)(generator_input)
x=layers.LeakyReLU()(x)
x=layers.Reshape((16,16,128))(x)
x=layers.Conv2D(256,5,padding='same')(x)
x=layers.LeakyReLU()(x)
x=layers.Conv2DTranspose(256,4,strides=2,padding='same')(x)
x=layers.LeakyReLU()(x)
x=layers.Conv2D(256,5,padding='same')(x)
x=layers.LeakyReLU()(x)
x=layers.Conv2D(256,5,padding='same')(x)
x=layers.LeakyReLU()(x)
x=layers.Conv2D(channels,7,activation='tanh',padding='same')(x)
generator=keras.models.Model(generator_input,x)
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

In [3]:
discriminator_input=layers.Input(shape=(height,width,channels))
x=layers.Conv2D(128,3)(discriminator_input)
x=layers.LeakyReLU()(x)
x=layers.Conv2D(128,4,strides=2)(x)
x=layers.LeakyReLU()(x)
x=layers.Conv2D(128,4,strides=2)(x)
x=layers.LeakyReLU()(x)
x=layers.Conv2D(128,4,strides=2)(x)
x=layers.LeakyReLU()(x)
x=layers.Flatten()(x)
x=layers.Dropout(0.4)(x)
x=layers.Dense(1,activation='sigmoid')(x)
discriminator=keras.models.Model(discriminator_input,x)
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 6, 6, 128)         0         
__________

In [4]:
discriminator_optimizer=keras.optimizers.RMSprop(lr=0.0008,clipvalue=1.0,decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer,loss='binary_crossentropy')

In [5]:
discriminator.trainable=False

In [6]:
gan_input=keras.Input(shape=(latent_dim,))
gan_output=discriminator(generator(gan_input))
gan=keras.models.Model(gan_input,gan_output)

gan_optimizer=keras.optimizers.RMSprop(lr=0.0004,clipvalue=1.0,decay=1e-8)
gan.compile(optimizer=gan_optimizer,loss='binary_crossentropy')

In [7]:
(x_train,y_train),(_,_)=cifar10.load_data()

In [8]:
a=y_train.flatten()==6
index=[]
for i in range(len(a)):
    if a[i]==True:
        index.append(i)

In [9]:
x_train=x_train[index]
x_train=x_train.reshape((x_train.shape[0],)+(height,width,channels)).astype('float32')/255

In [10]:
iterations=10000
batch_size=20
save_dir='./gan/'

start=0
for step in range(iterations):
    random_latent_vectors=np.random.normal(size=(batch_size,latent_dim))
    generated_images=generator.predict(random_latent_vectors)
    
    stop=start+batch_size
    real_images=x_train[start:stop]
    combined_images=np.concatenate([generated_images,real_images])
    
    labels=np.concatenate([np.ones((batch_size,1)),np.zeros((batch_size,1))])
    labels+=0.05*np.random.random(labels.shape)
    
    d_loss=discriminator.train_on_batch(combined_images,labels)
    random_latent_vectors=np.random.normal(size=(batch_size,latent_dim))
    misleading_tragets=np.zeros((batch_size,1))
    a_loss=gan.train_on_batch(random_latent_vectors,misleading_tragets)
    
    start+=batch_size
    if start>len(x_train)-batch_size:
        start=0
    if step%100==0:
        gan.save_weights('gan.h5')
        
        print('\nDiscriminator loss:',d_loss)
        print('\nAdverserial loss:',a_loss)
        
        img=image.array_to_img(generated_images[0]*255.,scale=False)
        img.save(os.path.join(save_dir,'generated'+str(step)+'.png'))
        
        img=image.array_to_img(real_images[0]*255.,scale=False)
        img.save(os.path.join(save_dir,'real'+str(step)+'.png'))

C:\Users\lucif\Anaconda3\envs\TF\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'



Discriminator loss: 0.6839183

Adverserial loss: 0.65192145

Discriminator loss: 0.5966033

Adverserial loss: 1.0690391

Discriminator loss: 0.76113045

Adverserial loss: 0.76456946

Discriminator loss: 0.69549596

Adverserial loss: 0.73518384

Discriminator loss: 0.6878458

Adverserial loss: 2.3250299

Discriminator loss: 0.6891195

Adverserial loss: 0.7491746

Discriminator loss: 0.6974033

Adverserial loss: 0.76907194

Discriminator loss: 0.7002711

Adverserial loss: 0.756145

Discriminator loss: 0.69077426

Adverserial loss: 0.722721

Discriminator loss: 0.6955525

Adverserial loss: 0.8556821

Discriminator loss: 0.706803

Adverserial loss: 0.85450995

Discriminator loss: 0.7024051

Adverserial loss: 0.7603878

Discriminator loss: 0.69503766

Adverserial loss: 0.7169459

Discriminator loss: 0.6829109

Adverserial loss: 0.76134723

Discriminator loss: 0.710065

Adverserial loss: 0.7339362

Discriminator loss: 0.71664083

Adverserial loss: 0.7738945

Discriminator loss: 0.7021867

A